In [2]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np

# 枫桥站

In [2]:
fengqiao = pd.read_csv('../data/intermediate/fengqiao_processed.csv', index_col=0)

mete = pd.read_csv('../data/intermediate/mete_processed.csv', index_col=0)

# 创建一个字典来存储观察数据
obs_data = {
    'pr': fengqiao['precipitation'],
    'ps': mete['PS'],
    'tas': mete['T2M'],
    'tasmax': mete['T2M_MAX'],
    'tasmin': mete['T2M_MIN'],
}

# Load the entire dictionary from the pickle file
with open('../data/intermediate/data_dict.pkl', 'rb') as f:
    loaded_data_dict = pickle.load(f)

scenarios = ['historical', 'ssp245', 'ssp585']
variables = ['pr', 'ps', 'tas', 'tasmax', 'tasmin']

for scenario in scenarios:
    for variable in variables:
        # Determine the start and end dates of the current dataset
        start_date = loaded_data_dict[scenario][variable].index.min()
        end_date = loaded_data_dict[scenario][variable].index.max()

        # Create a new date range that includes all dates between the start and end dates
        new_index = pd.date_range(start=start_date, end=end_date)

        # Reindex the dataframe to include any missing dates
        loaded_data_dict[scenario][variable] = loaded_data_dict[scenario][variable].reindex(new_index)

        # Interpolate to fill in the missing values
        loaded_data_dict[scenario][variable] = loaded_data_dict[scenario][variable].interpolate()

        if variable == 'pr':
            loaded_data_dict[scenario][variable] = loaded_data_dict[scenario][variable] * 86400 / 1000 *1000 
        elif variable == 'ps':
            loaded_data_dict[scenario][variable] = loaded_data_dict[scenario][variable] / 1000
        else:  # 'tas', 'tasmax', 'tasmin'
            loaded_data_dict[scenario][variable] = loaded_data_dict[scenario][variable] - 273.15

In [3]:
def predict_scenario_variables(obs,historical,future, scenario, variable):
# 如果它们不是DatetimeIndex，那么转换它们
    if not isinstance(obs.index, pd.DatetimeIndex):
        obs.index = pd.to_datetime(obs.index)
    if not isinstance(historical.index, pd.DatetimeIndex):
        historical.index = pd.to_datetime(historical.index) 
           
    # 寻找观测数据和模型预测数据的时间交集
    start_date = max(obs.index.min(), historical.index.min())
    end_date = min(obs.index.max(), historical.index.max())
    obs_overlap = obs.loc[start_date:end_date]
    historical_overlap = historical.loc[start_date:end_date]
# 在时间交集上选择观测数据和模型预测数据
    if variable == 'pr':
        future_corrected = linear_scaling_multiplicative(obs_overlap, historical_overlap, future)
    else:
        future_corrected = linear_scaling_additive(obs_overlap, historical_overlap, future)
    return future_corrected

def linear_scaling_multiplicative(obs, historical, future, percentile=99.97):
    # 计算模型历史偏差的百分位数阈值
    obs_threshold = np.percentile(obs, percentile)
    historical_threshold = np.percentile(historical, percentile)

    # 计算模型的历史偏差
    historical_ratio_high = obs[obs > obs_threshold].mean() / historical[historical > historical_threshold].mean()
    historical_ratio_low = obs[obs <= obs_threshold].mean() / historical[historical <= historical_threshold].mean()
    
    # 修正未来的偏差，根据阈值分别采用不同的偏差修正方法
    future_corrected = future.copy()
    future_corrected[future > historical_threshold] = future[future > historical_threshold] * historical_ratio_high
    future_corrected[future <= historical_threshold] = future[future <= historical_threshold] * historical_ratio_low
    return future_corrected


def linear_scaling_additive(obs, historical, future, percentile=99.97):
    # 计算模型历史偏差的百分位数阈值
    obs_threshold = np.percentile(obs, percentile)
    historical_threshold = np.percentile(historical, percentile)

    # 计算模型的历史偏差
    historical_bias_high = obs[obs > obs_threshold].mean() - historical[historical > historical_threshold].mean()
    historical_bias_low = obs[obs <= obs_threshold].mean() - historical[historical <= historical_threshold].mean()
    
    # 修正未来的偏差，根据阈值分别采用不同的偏差修正方法
    future_corrected = future.copy()
    future_corrected[future > historical_threshold] = future[future > historical_threshold] + historical_bias_high
    future_corrected[future <= historical_threshold] = future[future <= historical_threshold] + historical_bias_low
    return future_corrected


In [4]:
predictions = pd.DataFrame()

# 对 'ssp245' 和 'ssp585' 情景的变量进行预测
scenarios = ['ssp245', 'ssp585']
variables = ['pr', 'ps', 'tas', 'tasmax', 'tasmin']

for scenario in scenarios:
    for variable in variables:
        # 从字典中获取观察数据
        obs = obs_data[variable]

        # 从 loaded_data_dict 中获取历史数据和未来数据
        historical = loaded_data_dict['historical'][variable]
        future = loaded_data_dict[scenario][variable]

        # 调用函数进行预测
        future_corrected= predict_scenario_variables(obs, historical, future, scenario, variable)
        # 将预测结果添加到DataFrame中
        predictions[f'{scenario}_{variable}'] = future_corrected

In [5]:
predictions.round(2).to_csv('../data/intermediate/fengqiao_predictions_linear_scaling.csv')

# 苏州站

In [3]:
suzhou = pd.read_csv('../data/intermediate/suzhou_interpolated.csv', index_col=0)

mete = pd.read_csv('../data/intermediate/mete_processed.csv', index_col=0)

# 创建一个字典来存储观察数据
obs_data = {
    'pr': suzhou['precipitation'],
    'ps': mete['PS'],
    'tas': mete['T2M'],
    'tasmax': mete['T2M_MAX'],
    'tasmin': mete['T2M_MIN'],
}

# Load the entire dictionary from the pickle file
with open('../data/intermediate/data_dict.pkl', 'rb') as f:
    loaded_data_dict = pickle.load(f)

scenarios = ['historical', 'ssp245', 'ssp585']
variables = ['pr', 'ps', 'tas', 'tasmax', 'tasmin']

for scenario in scenarios:
    for variable in variables:
        # Determine the start and end dates of the current dataset
        start_date = loaded_data_dict[scenario][variable].index.min()
        end_date = loaded_data_dict[scenario][variable].index.max()

        # Create a new date range that includes all dates between the start and end dates
        new_index = pd.date_range(start=start_date, end=end_date)

        # Reindex the dataframe to include any missing dates
        loaded_data_dict[scenario][variable] = loaded_data_dict[scenario][variable].reindex(new_index)

        # Interpolate to fill in the missing values
        loaded_data_dict[scenario][variable] = loaded_data_dict[scenario][variable].interpolate()

        if variable == 'pr':
            loaded_data_dict[scenario][variable] = loaded_data_dict[scenario][variable] * 86400 / 1000 *1000 
        elif variable == 'ps':
            loaded_data_dict[scenario][variable] = loaded_data_dict[scenario][variable] / 1000
        else:  # 'tas', 'tasmax', 'tasmin'
            loaded_data_dict[scenario][variable] = loaded_data_dict[scenario][variable] - 273.15

In [4]:
def predict_scenario_variables(obs,historical,future, variable):
# 如果它们不是DatetimeIndex，那么转换它们
    if not isinstance(obs.index, pd.DatetimeIndex):
        obs.index = pd.to_datetime(obs.index)
    if not isinstance(historical.index, pd.DatetimeIndex):
        historical.index = pd.to_datetime(historical.index) 
           
    # 寻找观测数据和模型预测数据的时间交集
    start_date = max(obs.index.min(), historical.index.min())
    end_date = min(obs.index.max(), historical.index.max())
    obs_overlap = obs.loc[start_date:end_date]
    historical_overlap = historical.loc[start_date:end_date]
# 在时间交集上选择观测数据和模型预测数据
    if variable == 'pr':
        future_corrected = linear_scaling_multiplicative(obs_overlap, historical_overlap, future)
    else:
        future_corrected = linear_scaling_additive(obs_overlap, historical_overlap, future)
    return future_corrected

def linear_scaling_multiplicative(obs, historical, future, percentile=99.97):
    # 计算模型历史偏差的百分位数阈值
    obs_threshold = np.percentile(obs, percentile)
    historical_threshold = np.percentile(historical, percentile)

    # 计算模型的历史偏差
    historical_ratio_high = obs[obs > obs_threshold].mean() / historical[historical > historical_threshold].mean()
    historical_ratio_low = obs[obs <= obs_threshold].mean() / historical[historical <= historical_threshold].mean()
    
    # 修正未来的偏差，根据阈值分别采用不同的偏差修正方法
    future_corrected = future.copy()
    future_corrected[future > historical_threshold] = future[future > historical_threshold] * historical_ratio_high
    future_corrected[future <= historical_threshold] = future[future <= historical_threshold] * historical_ratio_low
    return future_corrected


def linear_scaling_additive(obs, historical, future, percentile=99.97):
    # 计算模型历史偏差的百分位数阈值
    obs_threshold = np.percentile(obs, percentile)
    historical_threshold = np.percentile(historical, percentile)

    # 计算模型的历史偏差
    historical_bias_high = obs[obs > obs_threshold].mean() - historical[historical > historical_threshold].mean()
    historical_bias_low = obs[obs <= obs_threshold].mean() - historical[historical <= historical_threshold].mean()
    
    # 修正未来的偏差，根据阈值分别采用不同的偏差修正方法
    future_corrected = future.copy()
    future_corrected[future > historical_threshold] = future[future > historical_threshold] + historical_bias_high
    future_corrected[future <= historical_threshold] = future[future <= historical_threshold] + historical_bias_low
    return future_corrected


In [5]:
predictions = pd.DataFrame()

# 对 'ssp245' 和 'ssp585' 情景的变量进行预测
scenarios = ['ssp245', 'ssp585']
variables = ['pr', 'ps', 'tas', 'tasmax', 'tasmin']

for scenario in scenarios:
    for variable in variables:
        # 从字典中获取观察数据
        obs = obs_data[variable]

        # 从 loaded_data_dict 中获取历史数据和未来数据
        historical = loaded_data_dict['historical'][variable]
        future = loaded_data_dict[scenario][variable]

        # 调用函数进行预测
        future_corrected= predict_scenario_variables(obs, historical, future, variable)
        # 将预测结果添加到DataFrame中
        predictions[f'{scenario}_{variable}'] = future_corrected

In [6]:
predictions.round(2).to_csv('../data/intermediate/suzhou_predictions_linear_scaling.csv')